<a href="https://colab.research.google.com/github/LukasStankevicius/Generating-abstractive-summaries-of-Lithuanian-news-articles-using-a-transformer-model/blob/main/Supplementary_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simple usage

In [ ]:
! pip install transformers

In [ ]:
from transformers import pipeline
name= "LukasStankevicius/t5-base-lithuanian-news-summaries-175"
my_pipeline = pipeline(task="text2text-generation", model=name, framework="pt")

Given the following article body from [15min](https://www.15min.lt/24sek/naujiena/lietuva/tarp-penkiu-rezultatyviausiu-tsrs-rinktines-visu-laiku-zaideju-trys-lietuviai-875-1380030):

In [ ]:
text = """
Latvijos krepšinio legenda Valdis Valteris pirmadienį socialiniame tinkle pasidalino statistika, kurios viršūnėje yra Arvydas Sabonis.
1982 metais TSRS rinktinėje debiutavęs 222 cm ūgio vidurio puolėjas su raudona apranga sužaidė 52 rungtynes, per kurias rinko po 15,6 taško. Tai pats aukščiausias rezultatyvumo vidurkis tarp visų sovietų komandai atstovavusių žaidėjų, skaičiuojant tuos, kurie sužaidė ne mažiau nei 50 rungtynių. Antras šioje rikiuotėje kitas buvęs Kauno „Žalgirio“ krepšininkas Rimas Kurtinaitis. Jis debiutavo TSRS rinktinėje vėliau nei Sabas, – 1984 metais, bet irgi sužaidė 52 mačus. R.Kurtinaitis pelnė po 15 taškų. 25-ių rezultatyviausių žaidėjų sąrašu pasidalinęs latvis V.Valteris, pelnęs po 13,8 taško, yra trečias.
Ketvirtas yra iš Kazachstano kilęs Valerijus Tichonenka, pelnęs po 13,7 taško per 79 rungtynes. Rezultatyviausią visų laikų TSRS rinktinės penketą uždaro Modestas Paulauskas. Lietuvos krepšinio legenda pelnė po 13,6 taško per 84 mačus.
Dešimtuke taip pat yra Oleksandras Volkovas (po 13,5 taško), Sergejus Belovas (12,7), Anatolijus Myškinas (po 12,3), Vladimiras Tkačenka (11,7) ir Aleksandras Salnikovas (11,4). Dvyliktas šiame sąraše yra Valdemaras Chomičius, vidutiniškai rinkęs po 10 taškų, o keturioliktas dar vienas buvęs žalgirietis Sergejus Jovaiša (po 9,8 taško). Šarūno Marčiulionio rezultatyvumo vidurkis turėjo būti aukštesnis, bet jis sužaidė mažiau nei 50 rungtynių. Kaip žinia, Lietuvai išsilaisvinus ir atkūrus Nepriklausomybę, visi minėti mūsų šalies krepšininkai, išskyrus karjerą jau baigusį M.Paulauską, užsivilko žalią aprangą ir atstovavo savo tėvynei.
A.Sabonis pagal rezultatyvumo vidurkį yra pirmas – jis Lietuvos rinktinei pelnė po 20 taškų. Antras pagal taškų vidurkį yra Artūras Karnišovas, rinkęs po 18,2 taško ir pelnęs iš viso daugiausiai taškų atstovaujant Lietuvos rinktinei (1453).
Tarp žaidėjų, kurie sužaidė bent po 50 oficialių rungtynių Lietuvos rinktinėje, trečią vietą užima Ramūnas Šiškauskas (po 12,9), ketvirtąją Linas Kleiza (po 12,7 taško), o penktas – Saulius Štombergas (po 11,1 taško). Daugiausiai rungtynių Lietuvos rinktinėje sužaidęs ir daugiausiai olimpinių medalių (3) su ja laimėjęs Gintaras Einikis rinko po 9,6 taško, o pirmajame trejete pagal rungtynių skaičių ir pelnytus taškus esantis Šarūnas Jasikevičius pelnė po 9,9 taško.
"""
text = ' '.join(text.strip().split())

The summary can be obtained by:

In [ ]:
my_pipeline(text)[0]["generated_text"]

#Advanced usage

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
name= "LukasStankevicius/t5-base-lithuanian-news-summaries-175"
tokenizer = T5Tokenizer.from_pretrained(name)
model = T5ForConditionalGeneration.from_pretrained(name)
def decode(x):
    return tokenizer.decode(x, skip_special_tokens=True)

Given the following article body from [15min](https://www.15min.lt/24sek/naujiena/lietuva/tarp-penkiu-rezultatyviausiu-tsrs-rinktines-visu-laiku-zaideju-trys-lietuviai-875-1380030):

In [ ]:
text = """
Latvijos krepšinio legenda Valdis Valteris pirmadienį socialiniame tinkle pasidalino statistika, kurios viršūnėje yra Arvydas Sabonis.
1982 metais TSRS rinktinėje debiutavęs 222 cm ūgio vidurio puolėjas su raudona apranga sužaidė 52 rungtynes, per kurias rinko po 15,6 taško. Tai pats aukščiausias rezultatyvumo vidurkis tarp visų sovietų komandai atstovavusių žaidėjų, skaičiuojant tuos, kurie sužaidė ne mažiau nei 50 rungtynių. Antras šioje rikiuotėje kitas buvęs Kauno „Žalgirio“ krepšininkas Rimas Kurtinaitis. Jis debiutavo TSRS rinktinėje vėliau nei Sabas, – 1984 metais, bet irgi sužaidė 52 mačus. R.Kurtinaitis pelnė po 15 taškų. 25-ių rezultatyviausių žaidėjų sąrašu pasidalinęs latvis V.Valteris, pelnęs po 13,8 taško, yra trečias.
Ketvirtas yra iš Kazachstano kilęs Valerijus Tichonenka, pelnęs po 13,7 taško per 79 rungtynes. Rezultatyviausią visų laikų TSRS rinktinės penketą uždaro Modestas Paulauskas. Lietuvos krepšinio legenda pelnė po 13,6 taško per 84 mačus.
Dešimtuke taip pat yra Oleksandras Volkovas (po 13,5 taško), Sergejus Belovas (12,7), Anatolijus Myškinas (po 12,3), Vladimiras Tkačenka (11,7) ir Aleksandras Salnikovas (11,4). Dvyliktas šiame sąraše yra Valdemaras Chomičius, vidutiniškai rinkęs po 10 taškų, o keturioliktas dar vienas buvęs žalgirietis Sergejus Jovaiša (po 9,8 taško). Šarūno Marčiulionio rezultatyvumo vidurkis turėjo būti aukštesnis, bet jis sužaidė mažiau nei 50 rungtynių. Kaip žinia, Lietuvai išsilaisvinus ir atkūrus Nepriklausomybę, visi minėti mūsų šalies krepšininkai, išskyrus karjerą jau baigusį M.Paulauską, užsivilko žalią aprangą ir atstovavo savo tėvynei.
A.Sabonis pagal rezultatyvumo vidurkį yra pirmas – jis Lietuvos rinktinei pelnė po 20 taškų. Antras pagal taškų vidurkį yra Artūras Karnišovas, rinkęs po 18,2 taško ir pelnęs iš viso daugiausiai taškų atstovaujant Lietuvos rinktinei (1453).
Tarp žaidėjų, kurie sužaidė bent po 50 oficialių rungtynių Lietuvos rinktinėje, trečią vietą užima Ramūnas Šiškauskas (po 12,9), ketvirtąją Linas Kleiza (po 12,7 taško), o penktas – Saulius Štombergas (po 11,1 taško). Daugiausiai rungtynių Lietuvos rinktinėje sužaidęs ir daugiausiai olimpinių medalių (3) su ja laimėjęs Gintaras Einikis rinko po 9,6 taško, o pirmajame trejete pagal rungtynių skaičių ir pelnytus taškus esantis Šarūnas Jasikevičius pelnė po 9,9 taško.
"""
text = ' '.join(text.strip().split())
input_dict = tokenizer(text,  padding=True, return_tensors="pt", return_attention_mask=True)

And generation parameters ([documentation](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate), [explanation](https://github.com/huggingface/blog/blob/master/notebooks/02_how_to_generate.ipynb)):

In [ ]:
g_kwargs = dict(max_length=512, num_beams=10, no_repeat_ngram_size=2, early_stopping=True)

The summary can be obtained by:

In [ ]:
output = model.generate(**input_dict, **g_kwargs)
list(map(decode, output.tolist()))[0]

If you do a lot of compute you can take advantage of GPU. Obtain summary with:

In [ ]:
input_dict = {key:value.to("cuda:0") for key, value in input_dict.items()}
model = model.to("cuda:0")
output = model.generate(**input_dict, **g_kwargs)
list(map(decode, output.cpu().tolist()))[0]

# Automatic evaluation
We evaluated summaries with [ROUGE](https://www.aclweb.org/anthology/W04-1013/). It measures *n-gram* overlap between reference and generated texts. However, one should not completely trust it as the same meaning can be expressed by different words (*n-grams*). Yet it is almost the best we can do (automated and fast). Lithuanian language is quite rich with different word stem endings so we also "helped" ROUGE by stemming words.


In [ ]:
! pip install rouge-score PyStemmer

Combining the two:

In [ ]:
from rouge_score import rouge_scorer
import Stemmer

class MyStemmer:
    def __init__(self):
        self.stemmer = Stemmer.Stemmer('lithuanian')

    def stem(self, token):
        return self.stemmer.stemWord(token)


class MyRougeScorer(rouge_scorer.RougeScorer):
    # I rewrite init to have different stemmer
    def __init__(self, rouge_types, use_stemmer=False):
        self.rouge_types = rouge_types
        self._stemmer = MyStemmer() if use_stemmer else None

Now, given the gold reference and generated summary:

In [ ]:
ground_truth = "Kai Lietuva dar buvo okupuota ir mūsų šalies krepšininkai privalėjo žaisti TSRS rinktinėje, keli jų buvo ryškūs lyderiai."
generated_text = "Lietuvos krepšinio federacijos (LKF) prezidento Arvydo Sabonio rezultatyvumo vidurkis yra aukščiausias tarp visų Sovietų Sąjungos rinktinėje atstovavusių žaidėjų, skaičiuojant tuos, kurie sužaidė bent po 50 oficialių rungtynių."

Let's calculate ROUGE:

In [ ]:
rouge_types = ['rouge1', 'rouge2', 'rougeL']
scorer = MyRougeScorer(rouge_types, use_stemmer=True)
score = scorer.score(ground_truth, generated_text)
print({s:score[s].fmeasure for s in rouge_types})

We monitored training by calculating ROUGE for 4096 validation pairs and noticed that after 250000 training steps our model started to overfit.


# How we trained the tokenizer
First let's install required libraries:

In [ ]:
! pip install sentencepiece protobuf

Now we need a very big text file. Suppose we have one with over 1000000 lines in it and name it `"my_big_text_file.txt"`. Be warned that the following code requires a lot of memory (you can reduce number of lines sampled by lowering `input_sentence_size`) and can take several hours.

In [ ]:
import sentencepiece as spm

default_kwargs = {
    "model_type": 'unigram', "pad_id": 0, "eos_id": 1, "unk_id": 2, "bos_id": -1, "pad_piece": '<pad>',
    "eos_piece": '</s>',
    "unk_piece": '<unk>', "input_sentence_size": 1000000, "max_sentencepiece_length": 64, "add_dummy_prefix": True
}
# more options are here: https://github.com/google/sentencepiece/blob/master/doc/options.md
spm.SentencePieceTrainer.train(
    input="my_big_text_file.txt",
    model_prefix="my_new_tokenizer",
    vocab_size=32000,
    split_by_whitespace=True,
    **default_kwargs
)
# normalization_rule_name=nmt_nfkc_cf if you want to lowercase

Now that our sentencepiece model is trained, let's put it in our `T5Tokenizer` from `transformers` library:

In [ ]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer("my_new_tokenizer.model", do_lower_case=False)
tokenizer._add_tokens(new_tokens=[f"<extra_id_{i}>" for i in range(100)] + ['</s>', '<pad>', '<unk>'],
                      special_tokens=True)
tokenizer.save_pretrained("MyNewT5Tokenizer")

So now you can load your trained tokenizer with:

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("MyNewT5Tokenizer")

# How we trained model?

## Optimizer and scheduler
We used [T5](https://arxiv.org/abs/1910.10683) transformer model. It was originally trained using [Adafactor](https://arxiv.org/abs/1804.04235) optimizer. We used it with with 10 000 warm-up steps followed by inverse square root internal learning rate schedule. Yet we still wanted to observe internal learning rate so we added some tweaks to original `transformers` library code. `Dummy` is for fake learning rate scheduler.

In [ ]:
from transformers import Adafactor, Trainer


class MyAdafactor(Adafactor):

    def __init__(self, args, **kwargs):
        super(MyAdafactor, self).__init__(args, **kwargs)
        self.last_learning_rates = []

    def step(self, closure=None):
        """
        Performs a single optimization step

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()
        all_lr = []
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.dtype in {torch.float16, torch.bfloat16}:
                    grad = grad.float()
                if grad.is_sparse:
                    raise RuntimeError("Adafactor does not support sparse gradients.")

                state = self.state[p]
                grad_shape = grad.shape

                factored, use_first_moment = self._get_options(group, grad_shape)
                # State Initialization
                if len(state) == 0:
                    state["step"] = 0

                    if use_first_moment:
                        # Exponential moving average of gradient values
                        state["exp_avg"] = torch.zeros_like(grad)
                    if factored:
                        state["exp_avg_sq_row"] = torch.zeros(grad_shape[:-1]).to(grad)
                        state["exp_avg_sq_col"] = torch.zeros(grad_shape[:-2] + grad_shape[-1:]).to(grad)
                    else:
                        state["exp_avg_sq"] = torch.zeros_like(grad)

                    state["RMS"] = 0
                else:
                    if use_first_moment:
                        state["exp_avg"] = state["exp_avg"].to(grad)
                    if factored:
                        state["exp_avg_sq_row"] = state["exp_avg_sq_row"].to(grad)
                        state["exp_avg_sq_col"] = state["exp_avg_sq_col"].to(grad)
                    else:
                        state["exp_avg_sq"] = state["exp_avg_sq"].to(grad)

                p_data_fp32 = p.data
                if p.data.dtype in {torch.float16, torch.bfloat16}:
                    p_data_fp32 = p_data_fp32.float()

                state["step"] += 1
                state["RMS"] = self._rms(p_data_fp32)
                lr = self._get_lr(group, state)

                beta2t = 1.0 - math.pow(state["step"], group["decay_rate"])
                update = (grad ** 2) + group["eps"][0]
                if factored:
                    exp_avg_sq_row = state["exp_avg_sq_row"]
                    exp_avg_sq_col = state["exp_avg_sq_col"]

                    exp_avg_sq_row.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-1))
                    exp_avg_sq_col.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-2))

                    # Approximation of exponential moving average of square of gradient
                    update = self._approx_sq_grad(exp_avg_sq_row, exp_avg_sq_col)
                    update.mul_(grad)
                else:
                    exp_avg_sq = state["exp_avg_sq"]

                    exp_avg_sq.mul_(beta2t).add_(1.0 - beta2t, update)
                    update = exp_avg_sq.rsqrt().mul_(grad)

                update.div_((self._rms(update) / group["clip_threshold"]).clamp_(min=1.0))
                update.mul_(lr)

                if use_first_moment:
                    exp_avg = state["exp_avg"]
                    exp_avg.mul_(group["beta1"]).add_(1 - group["beta1"], update)
                    update = exp_avg

                if group["weight_decay"] != 0:
                    p_data_fp32.add_(-group["weight_decay"] * lr, p_data_fp32)

                p_data_fp32.add_(-update)

                if p.data.dtype in {torch.float16, torch.bfloat16}:
                    p.data.copy_(p_data_fp32)

                all_lr.append(lr)
        self.last_learning_rates = torch.hstack(all_lr).cpu().numpy().tolist()
        return loss


class Dummy:
    def step(self):
        return 1

    def get_last_lr(self):
        return [1]

    def state_dict(self):
        return {"dummy_key": 1}

    def load_state_dict(self, state_dict):
        pass

    def get_lr(self):
        return [1]


class MyTrainer(Trainer):
      def _maybe_log_save_evaluate(self, tr_loss, model, trial, epoch):
        if self.control.should_log:
            logs: Dict[str, float] = {}
            tr_loss_scalar = tr_loss.item()
            # reset tr_loss to zero
            tr_loss -= tr_loss

            logs["loss"] = round(tr_loss_scalar / (self.state.global_step - self._globalstep_last_logged), 4)
            # backward compatibility for pytorch schedulers
            if self.args.adafactor:
                logs["learning_rate"] = self.optimizer.last_learning_rates
            else:
                logs["learning_rate"] = (
                    self.lr_scheduler.get_last_lr()[0]
                    if version.parse(torch.__version__) >= version.parse("1.4")
                    else self.lr_scheduler.get_lr()[0]
                )
            self._total_loss_scalar += tr_loss_scalar
            self._globalstep_last_logged = self.state.global_step

            self.log(logs)

        metrics = None
        if self.control.should_evaluate:
            metrics = self.evaluate()
            self._report_to_hp_search(trial, epoch, metrics)

        if self.control.should_save:
            self._save_checkpoint(model, trial, metrics=metrics)
            self.control = self.callback_handler.on_save(self.args, self.state, self.control)

## Data
Our training corpus consisted of over 6GB text file and was to big to load into the Colab RAM. So we:  
1) encoded it with our trained tokenizer - each string was converted to list of numbers from 0 to 32000;  
2) as our maximum number is 32000, we changed type of our lists to numpy arrays of type `uint16` which can contain integers from 0 to 65535;  
These "tricks" enabled us to load our pandas dataframe into Colab memory without memory errors.

In [ ]:
from torch.utils.data import DataLoader, Dataset

class My_Dataset(Dataset):
    def __init__(self, pickle_path):
        df = pd.read_pickle(pickle_path)
        self.examples = list(zip(df["main"], df["summary"]))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

class MyCollator2:
    """
This collator is used for already encoded strings. It only truncates and pads
    """

    def __init__(self, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, list_of_tuples):
        train_x, train_y = zip(*list_of_tuples)
        # truncate
        train_x, train_y = [seq[: self.max_length] for seq in train_x], [seq[: self.max_length] for seq in train_y]

        # first the targets
        n_items = len(train_y)
        tt = self.tokenizer.pad({"input_ids": train_y}, padding=True,
                                return_tensors="pt", return_attention_mask=True)

        decoder_input_ids = torch.cat((torch.zeros(size=(n_items, 1), dtype=torch.int64), tt['input_ids']), axis=1)
        decoder_attention_mask = torch.cat((torch.ones(size=(n_items, 1), dtype=torch.int64), tt['attention_mask']),
                                           axis=1)

        decoder_input_ids = decoder_input_ids[:, :-1]  # one item is added at beginning, so one at the end to remove
        decoder_attention_mask = decoder_attention_mask[:, :-1]

        # now inputs
        inputs_dict = self.tokenizer.pad({"input_ids": train_x},  padding=True, return_tensors="pt",
                                         return_attention_mask=True)
        # finally combine the two
        return {"decoder_input_ids": decoder_input_ids, "decoder_attention_mask": decoder_attention_mask,
                "labels": tt['input_ids'], **inputs_dict}

## Final training script

In [ ]:
from transformers import TrainingArguments 


kwargs = TrainingArguments(
    fp16=True, per_device_train_batch_size=4, gradient_accumulation_steps=32,
    num_train_epochs=30, seed=42, output_dir=output_dir, 
    evaluation_strategy="steps", per_device_eval_batch_size=4, 
    eval_accumulation_steps=None, max_grad_norm=None,
    lr_scheduler_type=SchedulerType.CONSTANT, warmup_steps=0,
    logging_first_step=True, logging_steps=2000, save_steps=5000, 
    dataloader_drop_last=True, eval_steps=2000, dataloader_num_workers=2, 
    deepspeed=None, adafactor=True
)


tokenizer = T5Tokenizer.from_pretrained("MyNewT5Tokenizer")
ds_train = My_Dataset("my_pandas_train_dataframe_pickle.gz")
ds_eval = My_Dataset("my_pandas_valid_dataframe_pickle.gz")

model = T5ForConditionalGeneration.from_pretrained("t5-base")
trainer = MyTrainer(
    model=model, data_collator=MyCollator2(tokenizer), tokenizer=tokenizer,
    args=kwargs, train_dataset=ds_train, eval_dataset=ds_eval,
    optimizers=(MyAdafactor((param for param in model.parameters() if param.requires_grad),
                            relative_step=True, warmup_init=True), Dummy()))

trainer.remove_callback(TensorBoardCallback)
trainer.train(resume_from_checkpoint=None)
trainer.save_model("output_directory_for_my_model")
trainer.state.save_to_json("output_directory_for_my_model/trainer_state.json")